In [1]:
import numpy as np
import pandas as pd
import ast
import datetime

In [2]:
scraping_date = '_26_04_2021'
df = pd.read_csv('data/full_profiles'+scraping_date+'.csv')

# Prepare variables for the analysis

## Format the last active date

In [3]:
df['last_active'] = pd.to_datetime(df.last_active, dayfirst=True)

## Remove useless features

In [4]:
df = df.drop(['description', 'city'], axis=1)

## Format the gender

In [5]:
df = df.replace({'gender':{'male':0, 'female':1, 'unknown':2}})

In [6]:
male = (df.gender==0).astype(int)
female = (df.gender==1).astype(int)
unknown = (df.gender==2).astype(int)

In [7]:
df.drop('gender',axis=1)
df['male'] = male
df['female'] = female
df['unknown'] = unknown

## Create dummy variables for the country

Start by removing the space at the beginning of the country string, then creating the dummy variables

In [8]:
df['country'] = df['country'].apply(lambda x: x.strip())

In [9]:
country_dummies = pd.get_dummies(df.country)

In [10]:
old_cols = list(country_dummies.columns)
country_dummies.columns = [country+'_country' for country in old_cols]

## Create dummy variables for the skills

In [11]:
available_skill = {}
skills = np.array(df.skills)
for profile_skill in skills:
    profile_skill = ast.literal_eval(profile_skill)
    if len(profile_skill)>0:
        for skill in profile_skill:
            if skill not in available_skill:
                available_skill[skill]=1
            else:
                available_skill[skill]+=1

In [12]:
sk = np.array(list(available_skill.keys()))
count_sk = np.array(list(available_skill.values()))
sorted_count_sk = np.argsort(-count_sk)
most_imp_sk = sk[sorted_count_sk]
top_100_sk = most_imp_sk[:100]

In [13]:
skills_dummies = pd.DataFrame()
for top_skill in top_100_sk:
    skills_dummies[top_skill+'_skill'] = df['skills'].apply(lambda x: top_skill in x).astype(int)

## Create dummy variables for the industry expertise

In [14]:
available_expertise = {}
industry_expertise = np.array(df.industry_expertise)
for profile_expertise in industry_expertise:
    profile_expertise = ast.literal_eval(profile_expertise)
    if len(profile_expertise)>0:
        for expertise in profile_expertise:
            if expertise not in available_expertise:
                available_expertise[expertise] = 1
            else:
                available_expertise[expertise] += 1

In [15]:
expertises = np.array(list(available_expertise.keys()))

In [16]:
exper_dummies = pd.DataFrame()
for exper in expertises:
    exper_dummies[exper+'_exper'] = df['industry_expertise'].apply(lambda x: exper in x).astype(int)

## Create dummy variables for the languages spoken

In [17]:
available_languages = {}
languages = np.array(df.languages)
for profile_languages in languages:
    profile_languages = ast.literal_eval(profile_languages)
    if len(profile_languages)>0:
        for language in profile_languages:
            if language not in available_languages:
                available_languages[language] = 1
            else:
                available_languages[language] += 1

In [18]:
languages = np.array(list(available_languages.keys()))

In [19]:
languages_dummies = pd.DataFrame()
for lang in languages:
    languages_dummies[lang+'_lang'] = df['languages'].apply(lambda x: lang in x).astype(int)

# Merge all the dummy variables in one dataframe

In [20]:
data = pd.concat([df, country_dummies, skills_dummies, exper_dummies, languages_dummies], axis=1)

# Prepare the data for the regression

## Keep only the active profiles

### Remove profiles with no projects

In [21]:
data = data.drop(data.loc[data['nb_projects']==0].index)

### Keep recently active profiles

In [22]:
threshold = datetime.datetime.strptime('2021-03-26', '%Y-%m-%d')
data['last_active'] = (data.last_active>threshold).astype(int)

In [23]:
data = data.reset_index(drop=True)

## Remove string features

We have to keep the name for the panel data but not for the simple data set

In [24]:
data = data.drop(['name', 'languages', 'skills', 'industry_expertise', 'country'], axis=1)

## Remove columns with only 0s

In [25]:
unwanted = [cols for cols in data.columns if data[cols].sum()==0]

In [26]:
len(unwanted)

47

In [27]:
data = data.drop(unwanted, axis=1)

In [28]:
data

,rating,nb_reviews,Price/hour in USD,nb_projects,nb_buyers,last_active,gender,score,male,female,...,Sotho_lang,Zulu_lang,Kikuyu_lang,Tigrinya_lang,Nynorsk_lang,Twi_lang,Nyanja_lang,Oriya_lang,Akan_lang,Khmer_lang
0,4.9,4694,34.74,7672,4168,1,1,10.043,0,1,...,0,0,0,0,0,0,0,0,0,0
1,5.0,1190,138.95,1085,533,1,2,8.691,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4.9,598,20.00,721,376,1,2,7.983,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.9,135,27.79,140,100,1,2,6.496,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.9,554,22.23,404,293,1,0,7.907,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7068,5.0,1,11.12,1,1,0,2,1.792,0,0,...,0,0,0,0,0,0,0,0,0,0
7069,5.0,6,48.34,5,4,0,2,3.434,0,0,...,0,0,0,0,0,0,0,0,0,0
7070,4.5,2,13.29,2,2,1,0,2.303,1,0,...,0,0,0,0,0,0,0,0,0,1
7071,4.7,18,41.68,25,18,0,0,4.450,1,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
data.to_csv('data/dummies'+scraping_date+'.csv', index=False)